In [ ]:
# load data from .dat file (No necessary)
import pandas as pd

# Read the file
data = pd.read_csv('/Users/yiwengeng/Documents/PregnantPrj/Data/PTL071323 tab delimited.dat', sep='\t')

# Display the first few rows
data.info()

In [ ]:
# load data from .sav file (No necessary)
import pyreadstat

# Read the .sav file
df, metadata = pyreadstat.read_sav('/Users/yiwengeng/Documents/PregnantPrj/Data/PTL071323.sav')

# Display the first few rows
df.head(5)

In [ ]:
import pandas as pd
from scipy.stats import pearsonr
# Read the file
data = pd.read_excel('/Users/yiwengeng/Documents/PregnantPrj/Data/PLT_stats.xlsx',sheet_name='Summary')
# print(data.info())
# Display the first few rows
# Filter the data
# collect_down data in which Alldata == 1
filtered_data = data[data['Alldata'] == 1]
# filtered_data = data[data['Alldata'].isin([0, 1])]
# Calculate the Pearson correlation coefficient for 'column1' and 'column2'
## iloc[:,2:55] -- Oura Data
## iloc[:,56:65] -- Demo Data
selected_data = filtered_data.iloc[:,2:] #
# print(selected_data.info())
correlation_matrix = selected_data.corr()
# print(correlation_matrix)

In [ ]:
correlation_matrix = correlation_matrix.applymap(lambda x: 1 if x >= 0.75 else (-1 if x <= -0.75 else x))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# Generate a heatmap
plt.figure(figsize=(20, 10), dpi=256)
# g = sns.heatmap(data=correlation_matrix, annot=True, fmt='d',cmap = "Blues", annot_kws={"size": 16})
# g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize = 16)

# Getting the Upper Triangle of the co-relation matrix
mask = np.zeros_like(correlation_matrix, dtype=np.bool_)
mask[np.tril_indices_from(mask)] = True

# using the upper triangle matrix as mask 
hmap = sns.heatmap(correlation_matrix, annot=False, mask=mask,cmap=sns.diverging_palette(220, 220, s=70, as_cmap=True)) #, annot_kws={"size": 16}
hmap.set_xticklabels(hmap.get_xticklabels(),rotation=90,fontsize=12)
hmap.set_yticklabels(hmap.get_yticklabels(),fontsize=12)
plt.show()

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
temp = list(selected_data.keys())
# newlist = temp[2:52]+temp[54:]
formula = 'eACEStotal ~ ' + ' + '.join(temp[0]+:])
print(formula)

In [ ]:
# VIF: multicollinearity -- not work
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

x = selected_data[['Std_activity_score', 'Std_bpm', 'Avg_5min_hr', 'Std_5min_hrv',
       'Avg_score_sleep_balance', 'Avg_score_activity_balance', 'DemoHeight']]
df_with_constant = sm.add_constant(x)

vif_data = pd.DataFrame()
vif_data["feature"] = df_with_constant.columns

# Calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(df_with_constant.values, i)
                          for i in range(len(df_with_constant.columns))]

print(vif_data)


# 递归，正则，

In [ ]:
x = selected_data[["Avg_sleep_score","Std_sleep_score",
                   "Avg_activity_score","Std_activity_score",
                   "Avg_readiness_score","Std_readiness_score",
                   "Avg_bpm","Std_bpm","Avg_5min_hr", "Std_5min_hr", 
                   "Avg_5min_hrv", "Std_5min_hrv", "Std_score_rem",
                   "Avg_score_sleep_balance","Avg_score_previous_day",
                   "Avg_score_activity_balance","Std_score_activity_balance",
                   "Avg_score_temperature",
                   "DemoEthn","DemoRace","DemoHeight","DemoWeight","DemoPlan",
                   "DemoMed","DemoCVEver","DemoMHDx","DemoInc","DemoEd"]]
y = selected_data["eACEStotal"]

In [ ]:
# Recursive Feature Elimination
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

model = LinearRegression()
rfe = RFE(model,n_features_to_select=7,step=1)
X_rfe = rfe.fit_transform(x,y)  
model.fit(X_rfe,y)
print(rfe.support_)
print(rfe.ranking_)
# please choose rfe.ranking_ from x
selected_features = x.columns[rfe.support_]


In [ ]:
x = selected_data[['Std_activity_score', 'Std_bpm', 'Avg_5min_hr', 'Std_5min_hrv',
       'Avg_score_sleep_balance', 'Avg_score_activity_balance', 'DemoHeight']]
y = selected_data["eACEStotal"]
X = sm.add_constant(x)
est = sm.OLS(y,X).fit()
est = smf.ols(formula='eACEStotal ~ Std_activity_score+Std_bpm+Avg_5min_hr+Std_5min_hrv+\
       Avg_score_sleep_balance+Avg_score_activity_balance+DemoHeight' , data=selected_data).fit()
print(est.summary())

In [ ]:
# Lasso Regularization
from sklearn.linear_model import LassoCV
reg = LassoCV()
reg.fit(x, y)
# print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
coef = pd.Series(reg.coef_, index = x.columns)
imp_coef = coef.sort_values()
print(imp_coef)
selected_features = x.columns[reg.coef_ != 0]
# Use the mask to select the columns of x

print(selected_features)


In [ ]:
x = selected_data[['Avg_activity_score', 'Avg_bpm', 'Avg_5min_hrv', 
       'Std_score_rem', 'Avg_score_sleep_balance',
       'Avg_score_activity_balance', 'DemoWeight']]
y = selected_data["eACEStotal"]
X = sm.add_constant(x)
est = sm.OLS(y,X).fit()
est = smf.ols(formula='eACEStotal ~ Avg_activity_score+Avg_bpm+Avg_5min_hrv+\
              Std_score_rem+Avg_score_sleep_balance+\
              Avg_score_activity_balance+DemoWeight' , data=selected_data).fit()
print(est.summary())

In [ ]:
x = selected_data[["Avg_sleep_score","Std_sleep_score",
                   "Avg_activity_score","Std_activity_score",
                   "Avg_readiness_score","Std_readiness_score",
                   "Avg_bpm","Std_bpm","Avg_5min_hr", "Std_5min_hr", 
                   "Avg_5min_hrv", "Std_5min_hrv", "Std_score_rem",
                   "Avg_score_sleep_balance","Avg_score_previous_day",
                   "Avg_score_activity_balance","Std_score_activity_balance",
                   "Avg_score_temperature",
                   "DemoEthn","DemoRace","DemoHeight","DemoWeight","DemoPlan",
                   "DemoMed","DemoCVEver","DemoMHDx","DemoInc","DemoEd"]]
y = selected_data["eACEStotal"]

In [ ]:
# PCA
from sklearn.decomposition import PCA
import sklearn.preprocessing as preprocessing
x = preprocessing.StandardScaler().fit_transform(x)
pca = PCA(n_components=7)

# Fit the PCA model to your data and apply the dimensionality reduction. 
# This returns a new array where the data has been projected onto the principal components.
pca_data = pca.fit_transform(x)
pca_df = pd.DataFrame(pca_data, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7'])
# final_df = pd.concat([pca_df, pd.DataFrame(y.values, columns=["eACEStotal"])], axis = 1)
# You can also access the principal components, the amount of variance that each PCA explains, and the cumulative variance explained.
# print("Principal components:", pca.components_.shape)
# print("Explained variance:", pca.explained_variance_)
# print("Cumulative explained variance:", np.cumsum(pca.explained_variance_ratio_))


In [ ]:
final_df = pd.concat([pca_df, pd.DataFrame(y.values, columns=["eACEStotal"])], axis = 1)

x = pca_df
y = final_df["eACEStotal"]

X = sm.add_constant(x)
est = sm.OLS(y,X).fit()
est = smf.ols(formula='eACEStotal ~ PC1 + PC2 + PC3+PC4+PC5+PC6+PC7' , data=final_df).fit()
print(est.summary())

In [ ]:
x = selected_data[["Avg_sleep_score","Avg_readiness_score","Avg_activity_score","DemoEthn","DemoRace","DemoHeight","DemoWeight","DemoPlan","DemoMed","DemoCVEver","DemoMHDx","DemoInc","DemoEd"]]
y = selected_data["eACEStotal"]
X = sm.add_constant(x)
est = sm.OLS(y,X).fit()
est = smf.ols(formula='eACEStotal ~ Avg_sleep_score + Avg_readiness_score + Avg_activity_score + DemoEthn + DemoRace + DemoHeight + DemoWeight + DemoPlan + DemoMed + DemoCVEver + DemoMHDx + DemoInc + DemoEd', data=selected_data).fit()
print(est.summary())

In [ ]:
summary = est.summary()

# Convert the tables to dataframes
summary_df1 = pd.DataFrame(summary.tables[0].data[1:], columns=summary.tables[0].data[0])
summary_df2 = pd.DataFrame(summary.tables[1].data[1:], columns=summary.tables[1].data[0])
summary_df3 = pd.DataFrame(summary.tables[2].data[1:], columns=summary.tables[2].data[0])

# Save the dataframes to CSV
summary_df1.to_csv('summary_df1.csv', index=False)
summary_df2.to_csv('summary_df2.csv', index=False)
summary_df3.to_csv('summary_df3.csv', index=False)

In [ ]:
print(est.summary())

In [ ]:
import pandas as pd

# Create a DataFrame to summarize the OLS Regression Results
summary_ols = {
    "Dep. Variable": ["eACEStotal"],
    "Model": ["OLS"],
    "Method": ["Least Squares"],
    "Date": ["Wed, 19 Jul 2023"],
    "Time": ["17:04:09"],
    "No. Observations": [15],
    "Df Residuals": [1],
    "Df Model": [13],
    "Covariance Type": ["nonrobust"],
    "R-squared": [0.999],
    "Adj. R-squared": [0.992],
    "F-statistic": [138.9],
    "Prob (F-statistic)": [0.0663],
    "Log-Likelihood": [26.732],
    "AIC": [-25.46],
    "BIC": [-15.55],
    "Omnibus": [0.243],
    "Prob(Omnibus)": [0.886],
    "Skew": [0.230],
    "Prob(JB)": [0.886],
    "Kurtosis": [2.579],
    "Cond. No.": [1.98e+04]
}

# Create a DataFrame for the regression coefficients
coef = {
    "Variables": ["Intercept", "Avg_sleep_score", "Avg_readiness_score", "Avg_activity_score", "DemoEthn", "DemoRace", "DemoHeight", "DemoWeight", "DemoPlan", "DemoMed", "DemoCVEver", "DemoMHDx", "DemoInc", "DemoEd"],
    "coef": [61.4300, -0.1999, -0.6428, 0.2576, -7.7665, -0.4085, -0.0922, 0.0126, -1.2026, 6.1338, 5.7675, -2.4878, 0.6558, 0.4567],
    "std err": [3.525, 0.021, 0.037, 0.012, 0.424, 0.068, 0.027, 0.002, 0.159, 0.295, 0.353, 0.218, 0.110, 0.077],
    "t": [17.428, -9.422, -17.603, 22.143, -18.319, -5.984, -3.451, 8.206, -7.570, 20.789, 16.343, -11.395, 5.965, 5.894],
    "P>|t|": [0.036, 0.067, 0.036, 0.029, 0.035, 0.105, 0.180, 0.077, 0.084, 0.031, 0.039, 0.056, 0.106, 0.107],
    "0.025": [16.642, -0.470, -1.107, 0.110, -13.153, -1.276, -0.432, -0.007, -3.221, 2.385, 1.284, -5.262, -0.741, -0.528],
    "0.975": [106.217, 0.070, -0.179, 0.405, -2.380, 0.459, 0.247, 0.032, 0.816, 9.883, 10.252, 0.286, 2.053, 1.441]
}

# Create dataframes
summary_ols_df = pd.DataFrame(summary_ols)
coef_df = pd.DataFrame(coef)

# Save to csv
summary_ols_df.to_csv('summary_ols.csv', index=False)
coef_df.to_csv('coef.csv', index=False)

In [ ]:
x = selected_data[["Avg_sleep_score","Std_sleep_score",
                   "Avg_activity_score","Std_activity_score",
                   "Avg_readiness_score","Std_readiness_score",
                   "Avg_bpm","Std_bpm","Avg_5min_hr", "Std_5min_hr", 
                   "Avg_5min_hrv", "Std_5min_hrv", "Std_score_rem",
                   "Avg_score_sleep_balance","Avg_score_previous_day",
                   "Avg_score_activity_balance","Std_score_activity_balance",
                   "Avg_score_temperature",
                   "DemoEthn","DemoRace","DemoHeight","DemoWeight","DemoPlan",
                   "DemoMed","DemoCVEver","DemoMHDx","DemoInc","DemoEd"]]
y = selected_data["eACEStotal"]
X = sm.add_constant(x)
est = sm.OLS(y,X).fit()
est = smf.ols(formula='eACEStotal ~ Avg_sleep_score + Std_sleep_score + \
              Avg_activity_score + Std_activity_score + \
              Avg_readiness_score + Std_readiness_score + \
              Avg_bpm + Std_bpm + Avg_5min_hr + Std_5min_hr + \
              Avg_5min_hrv + Std_5min_hrv + Std_score_rem + \
              Avg_score_sleep_balance + Avg_score_previous_day + \
              Avg_score_activity_balance + Std_score_activity_balance + \
              Avg_score_temperature + \
              DemoEthn + DemoRace + DemoHeight + DemoWeight + DemoPlan + \
              DemoMed + DemoCVEver + DemoMHDx + DemoInc + DemoEd' , data=selected_data).fit()
print(est.summary())